# 04 - Data Preparation

This notebook will be used to do common data preprocessing steps and splitting for our data. This data will later be used other notebooks where we do more specific preprocessing steps for models like logistic regression, and XGBoost and LSTM.

In [ ]:
import sys
import os

# Making the project modules available in the notebook
root = os.path.abspath(os.path.join('../..'))
if root not in sys.path: sys.path.append(root)

In [ ]:
import pandas as pd
import numpy as np

import torch
from sklearn.impute import SimpleImputer

In [ ]:
csv_path = os.path.join(root, 'project/data/downtime_window_sequences.csv')
df = pd.read_csv(csv_path)

In [ ]:
df.head()

In [ ]:
df.groupby('group_id')['label'].unique().value_counts()

In [ ]:
df.shape

### Data Pre-processing

In our pre-processing steps, we will perform the following in the order listed below:

- Check for data leakage
- Remove highly correlated features
- Remove constant columns as they do not provide any information
- Check for missing values
- Remove features used to label the data, along with features hand picked by me that I believe are not useful
- Split the data into training, validation, and test sets using a 70-15-15 split 
- After splitting the data, we will perform the following steps:
    - Remove some features that are not useful for the model
    - Impute missing values using the median value of the training set
- Lastly, we will convert the data to numpy arrays and save the pre-processed data to for use in the next notebook

### Data Leakage

There are a few things to check for data leakage in our dataset, such as:
- Overlapping windows was handleded in the downtime window extractor, but we need to confirm that this implementation actually worked.
- Remove features that highly correlate with the features used to label the data. This is because if we use these features to train the model, it will learn to predict the labels based on these features instead of the actual features that we want to use.

Later when we split the data, we will also make sure that the training, validation, and test sets do not overlap in time. This is important because we want to make sure that the model is not trained on data that it will see in the future. 

*(Written at a later stage:)* Earlier results during my masters resulted in excellent validation accuracy. They were so good that I was suspicious of data leakage. I did some investigations and concluded that there was no data leakage. However, as I was writing my report i realised that my tests was not thorough enough. I had missinterpreted the results of the test and drawn a false conclusion. I had to redo the test and found that there was indeed data leakage which I fixed in the downtime window extractor. To make sure that this worked, I will go through all data sample, retrieve the min and max timestamps and the machine id. I will then check if there are duplicates. 

In [ ]:
min_max_timestamp_machine_list = [] # list of tuples (min_timestamp, max_timestamp, machine_id)

In [ ]:
def extract_min_max_timestamp_per_machine(sub_df):
    downtime_df = sub_df.loc[sub_df['status'] == 'downtime']
    
    min_timestamp = downtime_df['timestamp'].min()
    max_timestamp = downtime_df['timestamp'].max()
    machine_id = downtime_df['machine_id'].iloc[0]
    
    min_max_timestamp_machine_list.append((min_timestamp, max_timestamp, machine_id))

In [ ]:
grouped = df.groupby('group_id')
grouped.apply(extract_min_max_timestamp_per_machine)

In [ ]:
min_max_timestamp_machine_list_set = set(min_max_timestamp_machine_list)
len(min_max_timestamp_machine_list_set) == len(min_max_timestamp_machine_list)

Great! There are no overlapping windows in the data. 

#### Check for features correlated to the features used to label the data

When making the labels, we used the following features:
- program
- unitNum
- PartCountAct
- program_cmt
- subprogram_cmt
- Machine_state (not directly used, but should be checked)

Even if we remove the first five features from the data, we still need to check if the other features are correlated to these features. We will do this by checking the correlation matrix and removing any features that are highly correlated with these features. We will use a threshold of 0.8 to remove features that are highly correlated with the features used to label the data.

After doing that, we will also check if any columns are highly correlated to the label column. We will use the same threshold to remove features.

In [ ]:
timestamps_used_to_label_data = [
    'Machine_state_machine',
    'machine_shdr_program_Numeric',
    'machine_shdr_unitNum_Numeric',
    'machine_shdr_PartCountAct_Numeric',
    'machine_shdr_program_cmt',
    'machine_shdr_subprogram_cmt',
]

In [ ]:
leakage_df = df.copy(deep=True)

In [ ]:
object_columns = leakage_df.select_dtypes(include=['object']).columns
object_columns

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
leakage_df['Machine_state_machine'] = le.fit_transform(leakage_df['Machine_state_machine'].astype(str))
leakage_df['machine_shdr_execution'] = le.fit_transform(leakage_df['machine_shdr_execution'].astype(str))

In [ ]:
def check_correlation(col_to_check, col_to_check_against):
    df_num = leakage_df[col_to_check + col_to_check_against]
    corr_all = df_num.corr()
    corr_block = corr_all.loc[col_to_check, col_to_check_against]
    max_corr = corr_block.abs().max(axis=1).sort_values(ascending=False)
    
    print(f'Top 10 correlations:')
    print(max_corr.head(10))

In [ ]:
# First we check the correlation between features used to label the data
feature_cols = [c for c in leakage_df.columns if c not in timestamps_used_to_label_data]

numeric_feature_cols = leakage_df[feature_cols].select_dtypes(include=[np.number]).columns.tolist()
numeric_label_cols = leakage_df[timestamps_used_to_label_data].select_dtypes(include=[np.number]).columns.tolist()

check_correlation(numeric_feature_cols, numeric_label_cols)

In [ ]:
# Secondly, we will check the correlation between the features and the label
leakage_df['label'] = le.fit_transform(leakage_df['label'].astype(str))

feature_cols = [c for c in leakage_df.columns if c != 'label']
numeric_feature_cols = leakage_df[feature_cols].select_dtypes(include=[np.number]).columns.tolist()

check_correlation(numeric_feature_cols, ['label'])

There does not seem to be any features that are highly correlated with either the features used to label the data or the label column.

### Remove constant columns

In [ ]:
constant_columns = df.columns[df.nunique() == 1]
constant_columns

In [ ]:
df.drop(columns=constant_columns, inplace=True)

In [ ]:
df.shape

### Check for missing values

In [ ]:
missing_values = df.isnull().mean() * 100
missing_values = missing_values.round(2)
missing_values

Some features have a lot of missing values. I believe the cause of this is either that the feature was not measured at an earlier stage or that the machine simply does not use the feature (take Zpos as an example, even though the machines have access to this feature, they may not use it that often). 

### Check for missing values per group

In [ ]:
percentage_missing_values_group = df.groupby('group_id').apply(lambda x: x.isnull().mean() * 100)

mean_percentage_missing_values_group = percentage_missing_values_group.mean(axis=1).round(2)
mean_percentage_missing_values_group_above_threshold = mean_percentage_missing_values_group[
    mean_percentage_missing_values_group > 50].sort_values(ascending=False) 

mean_percentage_missing_values_group_above_threshold

In [ ]:
len(mean_percentage_missing_values_group_above_threshold)

162 groups have about 80% of their data missing. This is such a high number that I will remove these groups from the data.

In [ ]:
groups_to_drop = mean_percentage_missing_values_group_above_threshold.index.tolist()

In [ ]:
df = df[~df['group_id'].isin(groups_to_drop)]
df.shape

In [ ]:
# we will run this code again to ensure that we have no more than 50% missing values in any group
percentage_missing_values_group = df.groupby('group_id').apply(lambda x: x.isnull().mean() * 100)

mean_percentage_missing_values_group = percentage_missing_values_group.mean(axis=1).round(2)
mean_percentage_missing_values_group_above_threshold = mean_percentage_missing_values_group[
    mean_percentage_missing_values_group > 50].sort_values(ascending=False) 

mean_percentage_missing_values_group_above_threshold

In [ ]:
df.groupby('group_id')['label'].unique().value_counts()

After removing the groups with a lot of missing values, we get a label distribution that is slightly unbalanced. This needs to be taken into account when training the model.

### Removing features

In [ ]:
df.columns

In [ ]:
features_to_drop = [
    # Features used to label the data
    'machine_shdr_program_Numeric',
    'machine_shdr_unitNum_Numeric',
    'machine_shdr_PartCountAct_Numeric',
    'machine_shdr_program_cmt',
    'machine_shdr_subprogram_cmt',
    
    # Features I deemed to be not useful
    'machine_shdr_subprogram_Numeric',
    'machine_shdr_Tool_suffix',
]

df.drop(columns=features_to_drop, inplace=True)
df.shape

There are still some features that needs to be dropped, but these will be dropped later.

### Split the data

Before splitting the data, we will encode the target labels and transform the data to numpy arrays. We will not do this with the features yet because we need the feature names to process the data after splitting.

In [ ]:
label_mapping = {'planned': 0, 'unplanned': 1}
df.loc[:, 'label'] = df['label'].map(label_mapping).astype(int)
df['label'].unique()

When splitting time series data into training, validation, and test sets, we need to make sure that the data is split in a way that preserves the temporal order of the data. This means that we cannot randomly split the data into training, validation, and test sets. Instead, we will use a time-based split where we will use the first 70% of the data for training, the next 15% for validation, and the last 15% for testing. This is an important step to prevent data leakage on time series data, because we want to make sure that the model is not trained on data that it will see in the future.

In [ ]:
def time_based_split(df, train_frac=0.7, val_frac=0.15):
    test_frac = 1.0 - train_frac - val_frac
    assert test_frac >= 0.0
    assert train_frac + val_frac <= 1.0
    assert train_frac + val_frac + test_frac == 1.0
    print(f'Train: {train_frac}, Val: {val_frac}, Test: {test_frac:.2f}')
    
    df = df.sort_values('start_time')
    
    # STRATIFIED SPLIT
    # train_parts, val_parts, test_parts = [], [], []
    
    # for label, group in df.groupby('label', sort=False):
    #     n = len(group)
    #     i_train = int(n * train_frac)
    #     i_val = int(n * (train_frac + val_frac))
        
    #     train_parts.append(group.iloc[:i_train])
    #     val_parts.append(group.iloc[i_train:i_val])
    #     test_parts.append(group.iloc[i_val:])
    
    # train_df = pd.concat(train_parts).sort_values('start_time').reset_index(drop=True)
    # val_df   = pd.concat(val_parts).sort_values('start_time').reset_index(drop=True)
    # test_df  = pd.concat(test_parts).sort_values('start_time').reset_index(drop=True)

    # NON-STRATIFIED SPLIT WITH NO OVERLAP
    train_df = df.iloc[:int(len(df) * train_frac)]
    val_df = df.iloc[int(len(df) * train_frac):int(len(df) * (train_frac + val_frac))]
    test_df = df.iloc[int(len(df) * (train_frac + val_frac)):]
    
    return train_df, val_df, test_df

In [ ]:
meta_data = (
    df.groupby('group_id')
    .agg(
        start_time = ('timestamp', 'min'),
        label = ('label', 'first')
    )
    .reset_index()
)
meta_data.head(10)

In [ ]:
train_meta, val_meta, test_meta = time_based_split(meta_data)

In [ ]:
train_meta.head(10)

In [ ]:
train_meta.tail(10)

In [ ]:
train_meta.shape, val_meta.shape, test_meta.shape

We have now split the data based on time, ensuring that the training, validation, and test sets do not overlap in time.

In [ ]:
print(train_meta['label'].value_counts())

Since we are using a time-based split, we cannot do proper stratification of the label distribution without overlapping the data. This imbalance will need to be handled later on.

In [ ]:
train_ids = train_meta['group_id']
val_ids = val_meta['group_id']
test_ids = test_meta['group_id']

In [ ]:
train_df = df[df['group_id'].isin(train_ids)].reset_index(drop=True)
val_df = df[df['group_id'].isin(val_ids)].reset_index(drop=True)
test_df = df[df['group_id'].isin(test_ids)].reset_index(drop=True)

In [ ]:
X_train = np.array([group.drop(columns=['label']) for _, group in train_df.groupby('group_id')], dtype=object)
X_val = np.array([group.drop(columns=['label']) for _, group in val_df.groupby('group_id')], dtype=object)
X_test = np.array([group.drop(columns=['label']) for _, group in test_df.groupby('group_id')], dtype=object)

y_train = train_df.groupby('group_id')['label'].first().values.astype(int)
y_val = val_df.groupby('group_id')['label'].first().values.astype(int)
y_test = test_df.groupby('group_id')['label'].first().values.astype(int)

In [ ]:
print(f'Train size:      {len(X_train)}')
print(f'Validation size: {len(X_val)}')
print(f'Test size:       {len(X_test)}')

In [ ]:
X_train[0].head(10)

In [ ]:
X_train[0].shape

In [ ]:
y_train[0]

### After we have splitted the data, we can remove the features that we do not need anymore

In [ ]:
X_train[0].columns

In [ ]:
def remove_featrues(dataset_list):
    features_to_remove = [
        'machine_id',
        'group_id',
    ]
    
    dataset = [group.drop(columns=features_to_remove) for group in dataset_list]
    return dataset

print(f'X_train shape before: {X_train[0].shape}')
X_train = remove_featrues(X_train)
print(f'X_train shape after: {X_train[0].shape}')

X_val = remove_featrues(X_val)
X_test = remove_featrues(X_test)

### Before we save our data we will impute missing values with the mean of the training set.

In [ ]:
X_train_concat_before_impute = pd.concat(X_train, axis=0)
print('Number of cells with missing values: ', X_train_concat_before_impute.isnull().sum().sum())

In [ ]:
numerical_columns = X_train_concat_before_impute.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = X_train_concat_before_impute.select_dtypes(include=['object']).columns

In [ ]:
numerical_imputer = SimpleImputer(strategy='mean').fit(X_train_concat_before_impute[numerical_columns])
categorical_imputer = SimpleImputer(strategy='most_frequent').fit(X_train_concat_before_impute[categorical_columns])

def impute_sequences(sub_df):
    sub_df[numerical_columns] = numerical_imputer.transform(sub_df[numerical_columns])
    sub_df[categorical_columns] = categorical_imputer.transform(sub_df[categorical_columns])
    return sub_df

X_train = [impute_sequences(group) for group in X_train]
X_val = [impute_sequences(group) for group in X_val]
X_test = [impute_sequences(group) for group in X_test]

In [ ]:
X_train_concat_after_imputation = pd.concat(X_train, axis=0)
print('Number of cells with missing values: ', X_train_concat_after_imputation.isnull().sum().sum())

We are now ready to save our datasets, ready to be further processed in the notebooks for logistic regression and XGBoost preprocessing and LSTM preprocessing.

In [ ]:
X_train[0].dtypes

In [ ]:
data_folder = os.path.join(root, 'project/data/')

torch.save({
    'X_train': X_train,
    'y_train': y_train,
    'X_val': X_val,
    'y_val': y_val,
    'X_test': X_test,
    'y_test': y_test,
    'numerical_imputer': numerical_imputer,
    'catergorical_imputer': categorical_imputer
}, data_folder + 'splitted_data.pt')